In [1]:
from gym import Env
import gym
from gym.spaces import Discrete, Box
import numpy as np
import random

In [2]:
#find the shortest path from state 0, to state 15
#create observation space using numpy array
#actions are 0=left, 1=down, 2=right, 3=up
#use the appropriate algorithm to learn
#print the optimal policy

class ShortestPath(Env):
    
    def __init__(self):
        self.observation_space = np.arange(16).reshape(4,4)
        self.action_space = Discrete(4)
        self.row = 0
        self.column = 0
        self.state = self.observation_space[self.row][self.column]
        
    def step(self, action):
        reward = -1
        
        #0 left
        if action == 0:
            if self.column!=0 :
                self.column -= 1  
        #1 down
        if action == 1:
            if self.row!=3:
                self.row += 1  
        #2 right
        if action == 2:
            if self.column!=3 :
                self.column += 1
        #3 up
        if action == 3:
            if self.row!=0:
                self.row -= 1
                
        new_state = self.observation_space[self.row][self.column]

        if new_state==15:
            done = True
            reward += 30
        else:
            done = False
            
        info = {}
        
        return new_state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        self.row = 0
        self.column = 0
        self.state = self.observation_space[self.row][self.column]
        return self.state

In [3]:
env = ShortestPath()
env.reset()

0

In [4]:
q_table = np.zeros((len(env.observation_space)*len(env.observation_space), env.action_space.n))

In [5]:
len(env.observation_space)*len(env.observation_space)

16

In [6]:
num_episodes=10000
epsilon=1
gamma = 1
learning_rate = 0.1

In [7]:
rewards= 0
rewardsAcrossEpisodes = []
#go through episodes
#at each episode we reset the environment, and don't forget to set done to False
for episode in range(num_episodes):
    state = env.reset()
    done = False
    r = 0
    
    while(True):
        #apply epsilon-greedy strategy
        num = random.uniform(0,1)
        if num < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state,:])
        #apply the chosen action to the environment
        new_state, reward, done, info = env.step(action)
        #update the value of the q_table according to: old_q = old_q + learning_rate*(reward + discount_factor*max(new_q) - old_q)
        q_table[state, action] += learning_rate*(reward + gamma * np.max(q_table[new_state, :]) - q_table[state, action])
        
        #update state to become next state
        #update reward
        state = new_state
        r += reward
        rewards += reward
        
        if done:
            rewardsAcrossEpisodes.append(r)
            epsilon = epsilon*0.9
            break

In [8]:
np.set_printoptions(suppress=True)
q_table

array([[-1.74484755, -1.71746132, 24.        , -1.73236612],
       [-1.10048599, 25.        , -1.08504825, -1.15455214],
       [-0.89483873, -0.09524088, -0.68441279, -0.69      ],
       [-0.56198031, -0.33345   , -0.5940931 , -0.5       ],
       [-1.19428805, -1.15981208, -0.71166481, -1.25799655],
       [-0.77295827, -0.80620844, 26.        , -0.90654347],
       [-0.48610608, 27.        , -0.33196689, -0.6145931 ],
       [-0.573553  ,  3.16769434, -0.29      , -0.292     ],
       [-0.7539    , -0.68294542, -0.67829747, -0.80757146],
       [-0.39729   , -0.1141344 ,  0.66616957, -0.571795  ],
       [-0.38371   ,  1.177579  , 28.        , -0.6797387 ],
       [ 1.13513416, 29.        ,  1.2457    , -0.072     ],
       [-0.38      , -0.81608177,  0.39279813, -0.462     ],
       [-0.5709459 , -0.2988    ,  5.05397194, -0.21041344],
       [ 0.24769503,  0.19      , 15.1293899 ,  0.33555719],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [9]:
#extract optimal policy
optimal_policy = []
for state in range(len(env.observation_space)*len(env.observation_space)):
    optimal_policy.append(np.argmax(q_table[state, :]))
    
optimal_policy

[2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 0]